In [ ]:
import sys

import rasterio as rio
import pandas as pd

import matplotlib.pyplot as plt
from tqdm import tqdm

from pathlib import Path

import numpy as np

### Config

In [ ]:
dataset_path = Path('../../change_detection_dataset/SN7_buildings/train')

train_df = '../../building_detection_dataset/dataset_2/train.csv'

### Function

### Create datasets

In [ ]:
df = pd.read_csv(train_df, index_col=0)
df = df[['AOI_name', 'filename']].drop_duplicates()

In [ ]:
df

In [ ]:
rgb_sum = np.array([0, 0, 0], dtype=np.uint64)
no_of_pixels = 0

for _, row in tqdm(df.iterrows()):
    AOI_name, file_name = row
    image_path = dataset_path / AOI_name / 'images_masked' / '{}.tif'.format(file_name)

    r = rio.open(image_path).read()

    image = r.transpose((1, 2, 0))[:, :, :-1]
    rgb_sum += image.sum(axis=(0, 1))

    no_of_pixels += (image != [0, 0, 0]).any(axis=-1).sum()

rgb_mean = rgb_sum / no_of_pixels
rgb_mean

In [ ]:
rgb_variance = np.array([0, 0, 0], dtype=np.float64)

for _, row in tqdm(df.iterrows()):
    AOI_name, file_name = row
    image_path = dataset_path / AOI_name / 'images_masked' / '{}.tif'.format(file_name)

    r = rio.open(image_path).read()

    image = r.transpose((1, 2, 0))[:, :, :-1]

    rgb_variance += (((image - rgb_mean) * (image != [0, 0, 0]).any(axis=-1)[:, :, None]) ** 2).sum(axis=(0, 1)) / no_of_pixels

rgb_variance

In [ ]:
std = rgb_variance ** 0.5
std

### checks

In [ ]:
aux_rgb_sum = np.array([0, 0, 0], dtype=np.float64)

for _, row in tqdm(df.iterrows()):
    AOI_name, file_name = row
    image_path = dataset_path / AOI_name / 'images_masked' / '{}.tif'.format(file_name)

    r = rio.open(image_path).read()

    image = r.transpose((1, 2, 0))[:, :, :-1]
    aux_rgb_sum += (((image - rgb_mean)*(image != [0, 0, 0]).any(axis=-1)[:, :, None])/std).sum(axis=(0, 1))

aux_rgb_mean = aux_rgb_sum / no_of_pixels
aux_rgb_mean

In [ ]:
aux_rgb_variance = np.array([0, 0, 0], dtype=np.float64)

for _, row in tqdm(df.iterrows()):
    AOI_name, file_name = row
    image_path = dataset_path / AOI_name / 'images' / '{}.tif'.format(file_name)

    r = rio.open(image_path).read()

    image = r.transpose((1, 2, 0))[:, :, :-1]

    aux_rgb_variance += ((((image - rgb_mean) / std) * (image != [0, 0, 0]).any(axis=-1)[:, :, None]) ** 2).sum(axis=(0, 1)) / no_of_pixels

aux_std = aux_rgb_variance ** 0.5
aux_std